# Imports

In [1]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
from xml.etree import ElementTree
import matplotlib.pyplot as plt
%matplotlib inline

# import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer

from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Left Align Tables in Jupyter Notebook
from IPython.core.display import HTML
table_css = 'table {align:left;display:block}'
HTML('<style>{}</style>'.format(table_css))

# ACQUIRE:

<div class="alert alert-success">

# Life Expectancy & Mortality across the Globe
* **GHO Selections:** https://apps.who.int/gho/athena/api/GHO
* **Homepage:** https://www.who.int/data/gho/info/athena-api-examples
* **CSV Data Pull:** https://apps.who.int/gho/athena/api/GHO/WHOSIS_000001,WHOSIS_000002,WHOSIS_000007,WHOSIS_000015?format=csv
    
<div class="alert alert-danger">

### Due to latency and timeout issues, the ```CSV Data Pull``` weblink above must be used to pull the CSV directly and save to a local file. Multiple refresh attempts may be required before getting the save file prompt.

In [3]:
# Read in csv from local file as leam_df (Life Expectancy And Mortality DataFrame)
leam_df = pd.read_csv('leam.csv')

In [4]:
# Check DataFrame
leam_df

,GHO,PUBLISHSTATE,YEAR,REGION,WORLDBANKINCOMEGROUP,COUNTRY,SEX,Display Value,Numeric,Low,High,StdErr,StdDev,Comments
0,WHOSIS_000002,PUBLISHED,2010,EMR,NaN,AFG,MLE,51.5,51.50902,NaN,NaN,NaN,NaN,NaN
1,WHOSIS_000015,PUBLISHED,2010,EMR,NaN,AFG,MLE,15.1,15.07067,NaN,NaN,NaN,NaN,NaN
2,WHOSIS_000007,PUBLISHED,2010,EMR,NaN,AFG,MLE,11.1,11.13451,NaN,NaN,NaN,NaN,NaN
3,WHOSIS_000001,PUBLISHED,2015,EMR,NaN,AFG,MLE,61.0,61.03658,NaN,NaN,NaN,NaN,NaN
4,WHOSIS_000015,PUBLISHED,2010,EMR,NaN,AFG,FMLE,15.2,15.16855,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9307,WHOSIS_000002,PUBLISHED,2010,GLOBAL,WB_WORLD_INCOME,NaN,FMLE,62.9,62.88848,NaN,NaN,NaN,NaN,NaN
9308,WHOSIS_000007,PUBLISHED,2000,GLOBAL,WB_WORLD_INCOME,NaN,BTSX,14.1,14.13883,NaN,NaN,NaN,NaN,NaN
9309,WHOSIS_000001,PUBLISHED,2010,GLOBAL,WB_WORLD_INCOME,NaN,BTSX,70.5,70.52787,NaN,NaN,NaN,NaN,NaN
9310,WHOSIS_000002,PUBLISHED,2010,GLOBAL,WB_WORLD_INCOME,NaN,BTSX,61.5,61.53046,NaN,NaN,NaN,NaN,NaN


In [5]:
leam_df.isnull().sum()

GHO                        0
PUBLISHSTATE               0
YEAR                       0
REGION                   192
WORLDBANKINCOMEGROUP    9072
COUNTRY                  528
SEX                        0
Display Value              0
Numeric                    0
Low                     9312
High                    9312
StdErr                  9312
StdDev                  9312
Comments                9312
dtype: int64

In [6]:
leam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9312 entries, 0 to 9311
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   GHO                   9312 non-null   object 
 1   PUBLISHSTATE          9312 non-null   object 
 2   YEAR                  9312 non-null   int64  
 3   REGION                9120 non-null   object 
 4   WORLDBANKINCOMEGROUP  240 non-null    object 
 5   COUNTRY               8784 non-null   object 
 6   SEX                   9312 non-null   object 
 7   Display Value         9312 non-null   float64
 8   Numeric               9312 non-null   float64
 9   Low                   0 non-null      float64
 10  High                  0 non-null      float64
 11  StdErr                0 non-null      float64
 12  StdDev                0 non-null      float64
 13  Comments              0 non-null      float64
dtypes: float64(7), int64(1), object(6)
memory usage: 1018.6+ KB


In [7]:
# List Columns
leam_df.columns

Index(['GHO', 'PUBLISHSTATE', 'YEAR', 'REGION', 'WORLDBANKINCOMEGROUP',
       'COUNTRY', 'SEX', 'Display Value', 'Numeric', 'Low', 'High', 'StdErr',
       'StdDev', 'Comments'],
      dtype='object')

<div class="alert alert-success">

# PREPARE:

<div class="alert alert-warning">

#### GHO Stats

In [8]:
# Show unique GHO Codes to ensure proper Data Pull
leam_df.GHO.unique()

array(['WHOSIS_000002', 'WHOSIS_000015', 'WHOSIS_000007', 'WHOSIS_000001'],
      dtype=object)

In [9]:
# Check GHO Null count
leam_df.GHO.isnull().sum()

0

In [10]:
# Check GHO value counts
leam_df.GHO.value_counts()

WHOSIS_000002    2328
WHOSIS_000015    2328
WHOSIS_000007    2328
WHOSIS_000001    2328
Name: GHO, dtype: int64

<div class="alert alert-warning">

#### WHOSIS Stats

<div class="alert alert-info">

#### These are all exactly the same record count at 2,328 entries
* 'WHOSIS_000001': 'Life expectancy at birth (years)', 
* 'WHOSIS_000002': 'Healthy life expectancy (HALE) at birth (years)', 
* 'WHOSIS_000007': 'Healthy life expectancy (HALE) at age 60 (years)', 
* 'WHOSIS_000015': 'Life expectancy at age 60 (years)', 

In [11]:
# Show percentage of entries for WHOSIS GHO Codes

print('These are all exactly the same record count at 2,328 entries')
print(f'WHOSIS_000001     {round(2328/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000002     {round(2328/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000007     {round(2328/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000015     {round(2328/leam_df.shape[0]*100,2)}%')

These are all exactly the same record count at 2,328 entries
WHOSIS_000001     25.0%
WHOSIS_000002     25.0%
WHOSIS_000007     25.0%
WHOSIS_000015     25.0%


#### WHOSIS Definitions:

* 'WHOSIS_000001': 'Life expectancy at birth (years)', 
* 'WHOSIS_000002': 'Healthy life expectancy (HALE) at birth (years)', 
* 'WHOSIS_000007': 'Healthy life expectancy (HALE) at age 60 (years)', 
* 'WHOSIS_000015': 'Life expectancy at age 60 (years)', 

#### WHOSIS Data Dictionary

|[GHO Code](https://apps.who.int/gho/athena/api/GHO)      |[Documentation](https://www.who.int/data/gho/indicator-metadata-registry)|[Global Health Observatory](https://www.who.int/data/gho) (GHO) Code Description|
|:-------------|:-----------:|:------------------------------------------------|
|WHOSIS_000001|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/65)|Life expectancy at birth (years)| 
|WHOSIS_000002|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/66)|Healthy life expectancy (HALE) at birth (years)| 
|WHOSIS_000007|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/3443)|Healthy life expectancy (HALE) at age 60 (years)| 
|WHOSIS_000015|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/2977)|Life expectancy at age 60 (years)| 

<div class="alert alert-warning">

# DROP Columns

In [12]:
# Show Null Count by Column in descending order
leam_df.isnull().sum().sort_values(ascending=False)

Low                     9312
High                    9312
StdErr                  9312
StdDev                  9312
Comments                9312
WORLDBANKINCOMEGROUP    9072
COUNTRY                  528
REGION                   192
GHO                        0
PUBLISHSTATE               0
YEAR                       0
SEX                        0
Display Value              0
Numeric                    0
dtype: int64

In [13]:
# Drop all columns with more than 10k Null and uneeded features
leam_df = leam_df.drop(columns=['Low',
                                'High',
                                'StdErr', 
                                'StdDev', 
                                'Comments', 
                                'WORLDBANKINCOMEGROUP'
                                ])

In [14]:
# Check work
leam_df

,GHO,PUBLISHSTATE,YEAR,REGION,COUNTRY,SEX,Display Value,Numeric
0,WHOSIS_000002,PUBLISHED,2010,EMR,AFG,MLE,51.5,51.50902
1,WHOSIS_000015,PUBLISHED,2010,EMR,AFG,MLE,15.1,15.07067
2,WHOSIS_000007,PUBLISHED,2010,EMR,AFG,MLE,11.1,11.13451
3,WHOSIS_000001,PUBLISHED,2015,EMR,AFG,MLE,61.0,61.03658
4,WHOSIS_000015,PUBLISHED,2010,EMR,AFG,FMLE,15.2,15.16855
...,...,...,...,...,...,...,...,...
9307,WHOSIS_000002,PUBLISHED,2010,GLOBAL,NaN,FMLE,62.9,62.88848
9308,WHOSIS_000007,PUBLISHED,2000,GLOBAL,NaN,BTSX,14.1,14.13883
9309,WHOSIS_000001,PUBLISHED,2010,GLOBAL,NaN,BTSX,70.5,70.52787
9310,WHOSIS_000002,PUBLISHED,2010,GLOBAL,NaN,BTSX,61.5,61.53046


In [15]:
# Show Null Count by Column in descending order
leam_df.isnull().sum().sort_values(ascending=False)

COUNTRY          528
REGION           192
GHO                0
PUBLISHSTATE       0
YEAR               0
SEX                0
Display Value      0
Numeric            0
dtype: int64

<div class="alert alert-warning">

#### Sex

In [16]:
# Show Null count for SEX
leam_df.SEX.isnull().sum()

0

<div class="alert alert-warning">

#### Country

In [17]:
# Show Null Count for COUNTRY
leam_df.COUNTRY.isnull().sum()

528

In [18]:
# Show unique number of Countries
leam_df.COUNTRY.nunique()

183

In [19]:
# Display Null percentage of entire DataFrame for COUNTRY
print(f'Country Null is {round(leam_df.COUNTRY.isnull().sum() / leam_df.shape[0] *100, 2)} % of entire DataFrame')

Country Null is 5.67 % of entire DataFrame


In [20]:
# List all unique Country values
leam_df.COUNTRY.unique()

array(['AFG', 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'ATG', 'AUS', 'AUT',
       'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS',
       'BIH', 'BLR', 'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA',
       'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG',
       'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CYP', 'CZE', 'DEU', 'DJI',
       'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH',
       'FIN', 'FJI', 'FRA', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIN',
       'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GTM', 'GUY', 'HND', 'HRV',
       'HTI', 'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR',
       'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR',
       'KOR', 'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO',
       'LTU', 'LUX', 'LVA', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MKD',
       'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI',
       'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NZL',
       'OMN', 'PAK',

In [21]:
# Create mask df for Null Country values
country_df = leam_df['COUNTRY'].isnull()

In [22]:
# Take a look
leam_df[country_df]

,GHO,PUBLISHSTATE,YEAR,REGION,COUNTRY,SEX,Display Value,Numeric
8333,WHOSIS_000007,PUBLISHED,2000,NaN,NaN,MLE,10.7,10.67575
8334,WHOSIS_000001,PUBLISHED,2010,NaN,NaN,MLE,58.2,58.18198
8335,WHOSIS_000002,PUBLISHED,2010,NaN,NaN,MLE,51.6,51.56751
8336,WHOSIS_000015,PUBLISHED,2010,NaN,NaN,MLE,15.2,15.24089
8337,WHOSIS_000001,PUBLISHED,2010,NaN,NaN,FMLE,62.1,62.06323
...,...,...,...,...,...,...,...,...
9307,WHOSIS_000002,PUBLISHED,2010,GLOBAL,NaN,FMLE,62.9,62.88848
9308,WHOSIS_000007,PUBLISHED,2000,GLOBAL,NaN,BTSX,14.1,14.13883
9309,WHOSIS_000001,PUBLISHED,2010,GLOBAL,NaN,BTSX,70.5,70.52787
9310,WHOSIS_000002,PUBLISHED,2010,GLOBAL,NaN,BTSX,61.5,61.53046


In [23]:
# List unique Region values in the Null country_df
leam_df[country_df].REGION.unique()

array([nan, 'AFR', 'AMR', 'SEAR', 'EUR', 'EMR', 'WPR', 'GLOBAL'],
      dtype=object)

In [24]:
# Show the Region value counts under the Null country_df
leam_df[country_df].REGION.value_counts()

AFR       48
AMR       48
SEAR      48
EUR       48
EMR       48
WPR       48
GLOBAL    48
Name: REGION, dtype: int64

In [25]:
# Show the Region value counts under the leam_df
leam_df.REGION.value_counts()

EUR       2448
AFR       2304
AMR       1632
EMR       1056
WPR       1056
SEAR       576
GLOBAL      48
Name: REGION, dtype: int64

<div class="alert alert-info">

* There are 528 Null values for country
    * That's less than 6% of the DataFrame
    * looking at the Null country_df Region values, Country is NaN for Global only stats
    * Looking at the leam_df, there is the same count of GLOBAL values meaning there are no other entries for GLOBAL REGION under COUNTRY besides NaN
    
#### Impute country NaN with GLOBAL where REGION equals GLOBAL 
#### DROP remaining NaN in country column

In [26]:
# IMPUTE NaN for [COUNTRY] with 'GLOBAL' if [REGION] == 'GLOBAL'
missing_mask = leam_df['COUNTRY'].isna()
mapping_dict = dict({'GLOBAL': 'GLOBAL'})
leam_df.loc[missing_mask, 'COUNTRY'] = leam_df.loc[missing_mask, 'REGION'].map(mapping_dict)
#leam_df['COUNTRY'] = leam_df['COUNTRY'].fillna('UNK')

In [27]:
# Create another mask for Null Country
country_df = leam_df['COUNTRY'].isnull()

In [28]:
# Check work
leam_df[country_df]

,GHO,PUBLISHSTATE,YEAR,REGION,COUNTRY,SEX,Display Value,Numeric
8333,WHOSIS_000007,PUBLISHED,2000,NaN,NaN,MLE,10.7,10.67575
8334,WHOSIS_000001,PUBLISHED,2010,NaN,NaN,MLE,58.2,58.18198
8335,WHOSIS_000002,PUBLISHED,2010,NaN,NaN,MLE,51.6,51.56751
8336,WHOSIS_000015,PUBLISHED,2010,NaN,NaN,MLE,15.2,15.24089
8337,WHOSIS_000001,PUBLISHED,2010,NaN,NaN,FMLE,62.1,62.06323
...,...,...,...,...,...,...,...,...
9295,WHOSIS_000007,PUBLISHED,2019,WPR,NaN,FMLE,17.7,17.69337
9296,WHOSIS_000001,PUBLISHED,2000,WPR,NaN,BTSX,72.4,72.42226
9297,WHOSIS_000002,PUBLISHED,2000,WPR,NaN,BTSX,64.3,64.26698
9298,WHOSIS_000015,PUBLISHED,2000,WPR,NaN,BTSX,19.3,19.31877


In [29]:
# ensure GLOBAL no longer has NULL country values
leam_df[country_df].REGION.unique()

array([nan, 'AFR', 'AMR', 'SEAR', 'EUR', 'EMR', 'WPR'], dtype=object)

In [30]:
# Show Null Count by Column in descending order
leam_df.isnull().sum().sort_values(ascending=False)

COUNTRY          480
REGION           192
GHO                0
PUBLISHSTATE       0
YEAR               0
SEX                0
Display Value      0
Numeric            0
dtype: int64

In [31]:
# Check Row Count
leam_df.shape[0]

9312

In [32]:
# Check effect of Dropping all NaN in DataFrame
leam_df.dropna().T

,0,1,2,3,4,5,6,7,8,9,...,9302,9303,9304,9305,9306,9307,9308,9309,9310,9311
GHO,WHOSIS_000002,WHOSIS_000015,WHOSIS_000007,WHOSIS_000001,WHOSIS_000015,WHOSIS_000007,WHOSIS_000001,WHOSIS_000002,WHOSIS_000007,WHOSIS_000001,...,WHOSIS_000007,WHOSIS_000001,WHOSIS_000015,WHOSIS_000007,WHOSIS_000001,WHOSIS_000002,WHOSIS_000007,WHOSIS_000001,WHOSIS_000002,WHOSIS_000015
PUBLISHSTATE,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,...,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED,PUBLISHED
YEAR,2010,2010,2010,2015,2010,2010,2015,2015,2010,2015,...,2000,2010,2000,2000,2010,2010,2000,2010,2010,2010
REGION,EMR,EMR,EMR,EMR,EMR,EMR,EMR,EMR,EMR,EMR,...,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL
COUNTRY,AFG,AFG,AFG,AFG,AFG,AFG,AFG,AFG,AFG,AFG,...,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL,GLOBAL
SEX,MLE,MLE,MLE,MLE,FMLE,FMLE,FMLE,FMLE,BTSX,BTSX,...,MLE,MLE,FMLE,FMLE,FMLE,FMLE,BTSX,BTSX,BTSX,BTSX
Display Value,51.5,15.1,11.1,61.0,15.2,10.7,62.3,52.4,10.9,61.7,...,13.1,68.0,20.4,15.1,73.2,62.9,14.1,70.5,61.5,20.1
Numeric,51.50902,15.07067,11.13451,61.03658,15.16855,10.71348,62.34584,52.44153,10.90975,61.65429,...,13.11078,67.97157,20.39618,15.10483,73.20403,62.88848,14.13883,70.52787,61.53046,20.13311


In [33]:
# Do the Math
leam_df.shape[0] - leam_df.dropna().shape[0]

480

In [34]:
# Check Percentage of dropping remainig NULL
print(f'Remaining Null/NaN equals {round(leam_df.dropna().shape[0] / leam_df.shape[0] - 1, 2) * -100} % of entries')

Remaining Null/NaN equals 5.0 % of entries


In [35]:
# DROP remaining Null
leam_df = leam_df.dropna()

In [36]:
# Take a look
leam_df

,GHO,PUBLISHSTATE,YEAR,REGION,COUNTRY,SEX,Display Value,Numeric
0,WHOSIS_000002,PUBLISHED,2010,EMR,AFG,MLE,51.5,51.50902
1,WHOSIS_000015,PUBLISHED,2010,EMR,AFG,MLE,15.1,15.07067
2,WHOSIS_000007,PUBLISHED,2010,EMR,AFG,MLE,11.1,11.13451
3,WHOSIS_000001,PUBLISHED,2015,EMR,AFG,MLE,61.0,61.03658
4,WHOSIS_000015,PUBLISHED,2010,EMR,AFG,FMLE,15.2,15.16855
...,...,...,...,...,...,...,...,...
9307,WHOSIS_000002,PUBLISHED,2010,GLOBAL,GLOBAL,FMLE,62.9,62.88848
9308,WHOSIS_000007,PUBLISHED,2000,GLOBAL,GLOBAL,BTSX,14.1,14.13883
9309,WHOSIS_000001,PUBLISHED,2010,GLOBAL,GLOBAL,BTSX,70.5,70.52787
9310,WHOSIS_000002,PUBLISHED,2010,GLOBAL,GLOBAL,BTSX,61.5,61.53046


In [37]:
# Check Work
leam_df.isnull().sum()

GHO              0
PUBLISHSTATE     0
YEAR             0
REGION           0
COUNTRY          0
SEX              0
Display Value    0
Numeric          0
dtype: int64

<div class="alert alert-warning">

# PIVOT

In [38]:
# Create df to check for duplicate records using columns that would create a unique index
dupe = leam_df[['YEAR','COUNTRY','SEX', 'GHO', 'Numeric']].duplicated(keep=False)

In [39]:
# Take a look (should be empty)
leam_df[dupe].T

""
GHO
PUBLISHSTATE
YEAR
REGION
COUNTRY
SEX
Display Value
Numeric


In [40]:
# Create pivot table
leam_pivot = leam_df.pivot_table(index=['YEAR','COUNTRY','SEX'], columns='GHO', values='Numeric')

In [41]:
# Take a look
leam_pivot

GHO                WHOSIS_000001  WHOSIS_000002  WHOSIS_000007  WHOSIS_000015
YEAR COUNTRY SEX                                                             
2000 AFG     BTSX       54.98949       46.76397       10.16798       13.92182
             FMLE       55.41726       46.58344       10.11293       14.20647
             MLE        54.57449       46.93113       10.22541       13.63259
     AGO     BTSX       49.30265       42.87378       10.73909       14.41537
             FMLE       52.12018       44.74432       11.80188       16.00562
...                          ...            ...            ...            ...
2019 ZMB     FMLE       65.36993       56.31194       13.65570       18.04779
             MLE        59.53606       52.49771       11.27271       14.68030
     ZWE     BTSX       60.68252       53.06544       11.48429       15.55015
             FMLE       63.60674       54.80482       12.43903       16.98877
             MLE        57.51101       51.20607       10.18165       13.58761

[2208 rows x 4 columns]

In [42]:
# assign pivot to df and reset index
leam_df = leam_pivot.reset_index()

In [43]:
# Check Work
leam_df.T

,0,1,2,3,4,5,6,7,8,9,...,2198,2199,2200,2201,2202,2203,2204,2205,2206,2207
GHO,,,,,,,,,,,,,,,,,,,,,
YEAR,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,...,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019
COUNTRY,AFG,AFG,AFG,AGO,AGO,AGO,ALB,ALB,ALB,ARE,...,YEM,ZAF,ZAF,ZAF,ZMB,ZMB,ZMB,ZWE,ZWE,ZWE
SEX,BTSX,FMLE,MLE,BTSX,FMLE,MLE,BTSX,FMLE,MLE,BTSX,...,MLE,BTSX,FMLE,MLE,BTSX,FMLE,MLE,BTSX,FMLE,MLE
WHOSIS_000001,54.98949,55.41726,54.57449,49.30265,52.12018,46.77004,73.54528,76.91914,70.65347,73.18027,...,64.41312,65.25417,68.28989,62.20109,62.4529,65.36993,59.53606,60.68252,63.60674,57.51101
WHOSIS_000002,46.76397,46.58344,46.93113,42.87378,44.74432,41.18177,65.23516,67.68733,63.14932,63.85107,...,56.90595,56.15329,57.69447,54.58585,54.40409,56.31194,52.49771,53.06544,54.80482,51.20607
WHOSIS_000007,10.16798,10.11293,10.22541,10.73909,11.80188,9.66284,14.98377,16.71034,13.50208,13.58815,...,12.77636,13.8575,14.772,12.69391,12.58347,13.6557,11.27271,11.48429,12.43903,10.18165
WHOSIS_000015,13.92182,14.20647,13.63259,14.41537,16.00562,12.82086,18.9965,21.2058,17.10085,18.68954,...,16.77037,19.12211,20.52139,17.34433,16.53152,18.04779,14.6803,15.55015,16.98877,13.58761


In [44]:
# check null
leam_df.isnull().sum().sort_values()

GHO
YEAR             0
COUNTRY          0
SEX              0
WHOSIS_000001    0
WHOSIS_000002    0
WHOSIS_000007    0
WHOSIS_000015    0
dtype: int64

<div class="alert alert-warning">

# Handle YEAR Column

In [45]:
# List unique year count
leam_df.YEAR.nunique()

4

In [46]:
# List year value counts
leam_df.YEAR.value_counts()

2000    552
2010    552
2015    552
2019    552
Name: YEAR, dtype: int64

In [47]:
# List unique year values
leam_df.YEAR.unique()

array([2000, 2010, 2015, 2019])

<div class="alert alert-info">

# Feature Engineer
* Replace Target Numeric Column that was lost during pivot
* Original Numeric Column was absorbed into new feature GHO Columns
* Create new Target Feature Life Expectancy column
    * Coloumn will equal mean of all Life Expectancy feature columns

<div class="alert alert-warning">

#### Life Expactancy in Years

* 'WHOSIS_000001': 'Life expectancy at birth (years)',
* 'WHOSIS_000002': 'Healthy life expectancy (HALE) at birth (years)',
* 'WHOSIS_000007': 'Healthy life expectancy (HALE) at age 60 (years)',
* 'WHOSIS_000015': 'Life expectancy at age 60 (years)',

In [48]:
# Create life_expectancy feature consisting of the mean of individual life expectancy features
leam_df['life_expectancy'] = leam_df[['WHOSIS_000001','WHOSIS_000002','WHOSIS_000007','WHOSIS_000015']].mean(axis=1)

In [49]:
leam_df

GHO,YEAR,COUNTRY,SEX,WHOSIS_000001,WHOSIS_000002,WHOSIS_000007,WHOSIS_000015,life_expectancy
0,2000,AFG,BTSX,54.98949,46.76397,10.16798,13.92182,31.460815
1,2000,AFG,FMLE,55.41726,46.58344,10.11293,14.20647,31.580025
2,2000,AFG,MLE,54.57449,46.93113,10.22541,13.63259,31.340905
3,2000,AGO,BTSX,49.30265,42.87378,10.73909,14.41537,29.332722
4,2000,AGO,FMLE,52.12018,44.74432,11.80188,16.00562,31.168000
...,...,...,...,...,...,...,...,...
2203,2019,ZMB,FMLE,65.36993,56.31194,13.65570,18.04779,38.346340
2204,2019,ZMB,MLE,59.53606,52.49771,11.27271,14.68030,34.496695
2205,2019,ZWE,BTSX,60.68252,53.06544,11.48429,15.55015,35.195600
2206,2019,ZWE,FMLE,63.60674,54.80482,12.43903,16.98877,36.959840


In [50]:
leam_df.life_expectancy.isnull().sum()

0

<div class="alert alert-danger">

#### IS THERE A PROBLEM WITH THE WAY THE MEAN IS BEING CALCULATED?

In [51]:
# Null/MIN/MAX of MEAN Life Expectancy in Years (ley)
leam_df.life_expectancy.isnull().sum(), leam_df.life_expectancy.min(), leam_df.life_expectancy.max()

(0, 18.1649, 53.206227500000004)

In [52]:
# MIN/MAX of Life expectancy at birth (years) 
leam_df.WHOSIS_000001.min(), leam_df.WHOSIS_000001.max()

(27.96683, 86.93782)

In [53]:
# MIN/MAX of Healthy life expectancy (HALE) at birth (years)
leam_df.WHOSIS_000002.min(), leam_df.WHOSIS_000002.max()

(25.37444, 75.48193)

In [54]:
# MIN/MAX of Healthy life expectancy (HALE) at age 60 (years)
leam_df.WHOSIS_000007.min(), leam_df.WHOSIS_000007.max()

(7.57875, 21.84938)

In [55]:
# MIN/MAX of Life expectancy at age 60 (years)
leam_df.WHOSIS_000015.min(), leam_df.WHOSIS_000015.max()

(10.14748, 28.55578)

In [56]:
# MEAN of all MAX
(leam_df.WHOSIS_000001.max() + leam_df.WHOSIS_000002.max() + leam_df.WHOSIS_000007.max() + leam_df.WHOSIS_000015.max()) / 4

53.206227500000004

In [57]:
# Including 0 or null into the mean
(25 + 0 + 0 + 25)/4

12.5

In [58]:
# Same input calculating mean without inclusion of 0
(25 + 0 + 0 + 25)/2

25.0

<div class="alert alert-info">

## NO

* This wa a problem on my previous Data Pull attempt because I was forced to impute Null with 0
* I could have negated the issue by getting the mean of all columns excluding 0 entries in the calculation
* At this point, I would not be able to feed the individual features but it would not be an issue since that would cause target leakage anyways
    
#### No issues, only notes on procedures learned from previous failed attempt

In [59]:
leam_df.shape[0]

2208

In [60]:
leam_df.isnull().sum()

GHO
YEAR               0
COUNTRY            0
SEX                0
WHOSIS_000001      0
WHOSIS_000002      0
WHOSIS_000007      0
WHOSIS_000015      0
life_expectancy    0
dtype: int64

In [61]:
leam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   YEAR             2208 non-null   int64  
 1   COUNTRY          2208 non-null   object 
 2   SEX              2208 non-null   object 
 3   WHOSIS_000001    2208 non-null   float64
 4   WHOSIS_000002    2208 non-null   float64
 5   WHOSIS_000007    2208 non-null   float64
 6   WHOSIS_000015    2208 non-null   float64
 7   life_expectancy  2208 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 138.1+ KB


<div class="alert alert-warning">

# Pivot & Encode SEX

In [62]:
leam_df.SEX.value_counts()

BTSX    736
FMLE    736
MLE     736
Name: SEX, dtype: int64

In [63]:
# Use pandas dummies to pivot features with more than two string values
# into multiple columns with binary int values that can be read as boolean
# drop_first = False in draft for human readability; Final will have it set to True.

dummy_df = pd.get_dummies(data=leam_df[['SEX']], drop_first=False)
dummy_df.T

,0,1,2,3,4,5,6,7,8,9,...,2198,2199,2200,2201,2202,2203,2204,2205,2206,2207
SEX_BTSX,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
SEX_FMLE,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
SEX_MLE,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1


In [64]:
# Check Value Counts
dummy_df.value_counts()

SEX_BTSX  SEX_FMLE  SEX_MLE
0         0         1          736
          1         0          736
1         0         0          736
dtype: int64

In [65]:
# Check Null Counts
dummy_df.isnull().sum()

SEX_BTSX    0
SEX_FMLE    0
SEX_MLE     0
dtype: int64

In [66]:
# Concat to leam DataFrame
leam_df = pd.concat([leam_df, dummy_df], axis=1)

In [67]:
# DROP original row for redundancy
leam_df = leam_df.drop(columns=['SEX'])

In [68]:
leam_df.T

,0,1,2,3,4,5,6,7,8,9,...,2198,2199,2200,2201,2202,2203,2204,2205,2206,2207
YEAR,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,...,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019
COUNTRY,AFG,AFG,AFG,AGO,AGO,AGO,ALB,ALB,ALB,ARE,...,YEM,ZAF,ZAF,ZAF,ZMB,ZMB,ZMB,ZWE,ZWE,ZWE
WHOSIS_000001,54.98949,55.41726,54.57449,49.30265,52.12018,46.77004,73.54528,76.91914,70.65347,73.18027,...,64.41312,65.25417,68.28989,62.20109,62.4529,65.36993,59.53606,60.68252,63.60674,57.51101
WHOSIS_000002,46.76397,46.58344,46.93113,42.87378,44.74432,41.18177,65.23516,67.68733,63.14932,63.85107,...,56.90595,56.15329,57.69447,54.58585,54.40409,56.31194,52.49771,53.06544,54.80482,51.20607
WHOSIS_000007,10.16798,10.11293,10.22541,10.73909,11.80188,9.66284,14.98377,16.71034,13.50208,13.58815,...,12.77636,13.8575,14.772,12.69391,12.58347,13.6557,11.27271,11.48429,12.43903,10.18165
WHOSIS_000015,13.92182,14.20647,13.63259,14.41537,16.00562,12.82086,18.9965,21.2058,17.10085,18.68954,...,16.77037,19.12211,20.52139,17.34433,16.53152,18.04779,14.6803,15.55015,16.98877,13.58761
life_expectancy,31.460815,31.580025,31.340905,29.332722,31.168,27.608877,43.190177,45.630653,41.10143,42.327258,...,37.71645,38.596767,40.319437,36.706295,36.492995,38.34634,34.496695,35.1956,36.95984,33.121585
SEX_BTSX,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
SEX_FMLE,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
SEX_MLE,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1


<div class="alert alert-warning">

# Cache Clean DataFrame

In [69]:
# Cache a Clean version of my data
leam_df.to_csv('clean_leam.csv')

In [70]:
# Check it was written
!ls *.csv

clean_leam.csv     leam.csv           leam2.csv          online_example.csv
